<a href="https://colab.research.google.com/github/Tonyblair2002/trail/blob/main/Mobile_Price_Range_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving sample_submission.csv to sample_submission.csv
User uploaded file "sample_submission.csv" with length 4215 bytes


In [7]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving test_data (1).csv to test_data (1).csv
User uploaded file "test_data (1).csv" with length 38647 bytes


In [8]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving train_data (1).csv to train_data (1).csv
User uploaded file "train_data (1).csv" with length 91622 bytes


In [10]:
import warnings  
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [17]:
train=pd.read_csv('test_data (1).csv')
test=pd.read_csv('train_data (1).csv')
a=pd.DataFrame(test['id'])

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             600 non-null    int64  
 1   battery_power  600 non-null    int64  
 2   bluetooth      600 non-null    int64  
 3   clock_speed    600 non-null    float64
 4   dual_sim       600 non-null    int64  
 5   fc             600 non-null    int64  
 6   four_g         600 non-null    int64  
 7   int_memory     600 non-null    int64  
 8   m_dep          600 non-null    float64
 9   mobile_wt      600 non-null    int64  
 10  n_cores        600 non-null    int64  
 11  pc             600 non-null    int64  
 12  px_height      600 non-null    int64  
 13  px_width       600 non-null    int64  
 14  ram            600 non-null    int64  
 15  sc_h           600 non-null    int64  
 16  sc_w           600 non-null    int64  
 17  talk_time      600 non-null    int64  
 18  three_g   

In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1400 non-null   int64  
 1   battery_power  1400 non-null   int64  
 2   bluetooth      1400 non-null   int64  
 3   clock_speed    1400 non-null   float64
 4   dual_sim       1400 non-null   int64  
 5   fc             1400 non-null   int64  
 6   four_g         1400 non-null   int64  
 7   int_memory     1400 non-null   int64  
 8   m_dep          1400 non-null   float64
 9   mobile_wt      1400 non-null   int64  
 10  n_cores        1400 non-null   int64  
 11  pc             1400 non-null   int64  
 12  px_height      1400 non-null   int64  
 13  px_width       1400 non-null   int64  
 14  ram            1400 non-null   int64  
 15  sc_h           1400 non-null   int64  
 16  sc_w           1400 non-null   int64  
 17  talk_time      1400 non-null   int64  
 18  three_g 

In [23]:
import seaborn as sns
import matplotlib.pyplot as plt
drops=['bluetooth','fc','m_dep','n_cores','talk_time','pc','clock_speed','mobile_wt','three_g']

In [24]:
min(train['ram'])

265

In [25]:
bb=['ram_cat','ram','battery','px_width','px_height','battery_power','sc_h','sc_w']
train.corr()

,id,battery_power,bluetooth,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
id,1.000000,0.034809,-0.030893,-0.064305,-0.043976,0.048467,0.017566,-0.002274,-0.022149,0.080820,-0.012976,0.015447,0.072581,0.097429,-0.058543,-0.019249,0.020739,0.085033,0.067387,0.084072,0.056843
battery_power,0.034809,1.000000,0.060853,0.006013,-0.050839,0.042487,0.019553,-0.054094,0.030657,-0.017481,-0.057251,0.004854,0.073838,-0.000195,-0.007701,-0.050076,-0.038294,0.050977,0.036485,-0.007155,-0.034702
bluetooth,-0.030893,0.060853,1.000000,-0.045148,0.081191,0.027763,0.055595,0.074653,0.048033,-0.000110,-0.018442,-0.012460,0.000451,-0.075356,0.036717,-0.011914,-0.013556,0.028760,-0.000319,-0.013239,-0.053660
clock_speed,-0.064305,0.006013,-0.045148,1.000000,-0.042924,0.035138,-0.102312,0.019474,-0.047055,0.004135,-0.028673,0.012023,-0.033237,-0.009700,0.056943,0.000106,-0.016153,-0.038839,-0.069925,-0.047361,-0.045465
dual_sim,-0.043976,-0.050839,0.081191,-0.042924,1.000000,-0.062369,-0.002575,-0.045681,-0.044379,-0.020293,-0.062430,-0.019979,-0.074517,-0.034157,0.009950,-0.025538,0.003932,-0.075134,0.003930,-0.086848,0.020381
fc,0.048467,0.042487,0.027763,0.035138,-0.062369,1.000000,-0.001569,-0.021072,0.029543,0.053192,0.002229,0.653767,-0.041223,-0.023465,0.040774,0.000464,-0.077393,-0.015161,-0.011337,-0.012880,0.015129
four_g,0.017566,0.019553,0.055595,-0.102312,-0.002575,-0.001569,1.000000,-0.021457,0.105550,-0.046353,0.028607,-0.034471,-0.048086,0.013464,0.088174,0.005755,0.044876,-0.019439,0.580712,0.006449,0.014069
int_memory,-0.002274,-0.054094,0.074653,0.019474,-0.045681,-0.021072,-0.021457,1.000000,0.013946,0.036916,-0.058792,-0.018216,0.090643,0.036624,0.036310,0.038826,0.031987,0.011529,-0.033984,-0.061561,0.026109
m_dep,-0.022149,0.030657,0.048033,-0.047055,-0.044379,0.029543,0.105550,0.013946,1.000000,0.017890,-0.009807,0.047529,-0.003579,0.006773,-0.006552,-0.040527,-0.021678,0.024021,0.052671,0.003004,-0.018393
mobile_wt,0.080820,-0.017481,-0.000110,0.004135,-0.020293,0.053192,-0.046353,0.036916,0.017890,1.000000,-0.025853,0.032020,-0.047219,0.002169,0.000824,-0.067321,-0.000935,0.087269,-0.019296,-0.014328,0.005130
